<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_online_0616/blob/main/tibame_night_20250708.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


```
HTTP代碼
1. 2: 成功
2. 3: 轉址
3. 4: 錯誤
403 Forbidden
1. IP被ban: 換ip/等
2. 模仿不夠像: 缺少必要headers
```



In [5]:
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1751962851.A.45A.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0")
resp = req.urlopen(r)
content = resp.read()
html = bs.BeautifulSoup(content)
print(html)
# find/find_all
# .text .get_text() ["href"]

# 作者/標題/時間/看板
row = {
    "author":
    "title":
    "post_time":
    "board":
    "pushes":[
        {
            "type":1推 -1噓 0->
            "uid":
            "text":
        }
    ]
}

SyntaxError: invalid syntax (ipython-input-5-3611974861.py, line 17)

In [ ]:
""